# 🚀 Day 1: Grid-Stride Loops - The Professional Pattern

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdodlapati3/cuda-lab/blob/main/learning-path/week-03/day-1-grid-stride-loops.ipynb)

---

## 🎯 The Challenge

*You've launched 1 million threads for 1 million elements. What happens when you have 10 million elements tomorrow?*

The naive "one thread per element" approach breaks down with real-world data sizes. Professional CUDA developers use a single elegant pattern that handles **any data size** with **fixed launch configurations**: the **grid-stride loop**.

---

## 📚 Learning Objectives

By the end of this session, you will be able to:

| Objective | Skill Level |
|-----------|-------------|
| Identify limitations of naive "one thread per element" approach | Understand |
| Implement grid-stride loops for arbitrary data sizes | Apply |
| Configure optimal thread blocks independent of problem size | Apply |
| Extend patterns to 1D and 2D grid-stride loops | Create |

---

## 🗺️ Session Roadmap

| Part | Topic | Duration |
|------|-------|----------|
| 1 | The Problem with Naive Kernels | 10 min |
| 2 | Grid-Stride Loop Pattern | 15 min |
| 3 | 2D Grid-Stride Loops | 15 min |
| 4 | Performance Analysis | 10 min |
| 5 | Exercises | 10 min |

> **Primary Focus:** CUDA C++ code examples first, Python/Numba backup for interactive testing

---

In [ ]:
# ⚙️ Colab/Local Setup - Run this first!
import subprocess, sys
try:
    import google.colab
    print("🔧 Running on Google Colab - Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
    print("✅ Setup complete!")
except ImportError:
    print("💻 Running locally - make sure you have: pip install numba numpy")

import numpy as np
from numba import cuda
import math
import time

# Verify GPU
print(f"\nCUDA available: {cuda.is_available()}")
if cuda.is_available():
    device = cuda.get_current_device()
    print(f"Device: {device.name}")
    print(f"Compute capability: {device.compute_capability}")
    print(f"Max threads per block: {device.MAX_THREADS_PER_BLOCK}")
    print(f"Max blocks per grid: {device.MAX_GRID_DIM_X}")

---

## Part 1: The Problem with Naive Kernels

> 💡 **Concept Card: The Scalability Problem**
> 
> ```
> 🎯 THE NAIVE APPROACH LIMITATION
> ═══════════════════════════════════════════════════════
> 
>   NAIVE: One Thread = One Element
>   
>   Thread 0 → Element 0    ✅ Works for small arrays
>   Thread 1 → Element 1    
>   Thread 2 → Element 2    ❌ But what if N > max_threads?
>   ...                     ❌ What if N changes at runtime?
>   
>   PROBLEM: Must recalculate grid dimensions for every N!
>   
>   blocks = (N + 255) / 256   ← Changes with data size
>   kernel<<<blocks, 256>>>()  ← Must relaunch differently
>   
> ═══════════════════════════════════════════════════════
> ```
> 
> **Why It Matters:** Production code processes varying data sizes. Reconfiguring launches is error-prone and inflexible.

### 🔷 CUDA C++ Implementation (Primary)

Compile and run:
```bash
nvcc -arch=sm_75 -o naive_add naive_add.cu
./naive_add
```

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
%%writefile naive_add.cu
// naive_add.cu - Naive vector addition
#include <stdio.h>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", \
                    __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

// Naive kernel: one thread handles exactly one element
__global__ void naiveAdd(const float* a, const float* b, float* out, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (tid < n) {  // Bounds check
        out[tid] = a[tid] + b[tid];
    }
}

int main() {
    int n = 1000000;
    size_t size = n * sizeof(float);
    
    // Allocate host memory
    float *h_a, *h_b, *h_out;
    h_a = (float*)malloc(size);
    h_b = (float*)malloc(size);
    h_out = (float*)malloc(size);
    
    // Initialize data
    for (int i = 0; i < n; i++) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
    }
    
    // Allocate device memory
    float *d_a, *d_b, *d_out;
    CUDA_CHECK(cudaMalloc(&d_a, size));
    CUDA_CHECK(cudaMalloc(&d_b, size));
    CUDA_CHECK(cudaMalloc(&d_out, size));
    
    // Copy to device
    CUDA_CHECK(cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice));
    
    // Launch configuration
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    
    printf("Launching with %d blocks, %d threads/block\n", blocksPerGrid, threadsPerBlock);
    
    naiveAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_out, n);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());
    
    // Copy back and verify
    CUDA_CHECK(cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost));
    printf("Result[0] = %f (expected 3.0)\n", h_out[0]);
    
    // Cleanup
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_out);
    free(h_a); free(h_b); free(h_out);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o naive_add naive_add.cu
!./naive_add

In [ ]:
# Python equivalent for interactive testing
@cuda.jit
def naive_add(a, b, out, n):
    """Naive: One thread handles exactly one element."""
    tid = cuda.grid(1)
    
    if tid < n:  # Bounds check
        out[tid] = a[tid] + b[tid]

---

## Part 2: Grid-Stride Loop Pattern

> 💡 **Concept Card: The Grid-Stride Loop Pattern**
> 
> ```
> 🔄 GRID-STRIDE: Handle ANY Size with FIXED Config
> ═══════════════════════════════════════════════════════
> 
>   Array: [0][1][2][3][4][5][6][7][8][9][10][11][12][13]
>                                                    
>   Grid has 4 threads total (e.g., 2 blocks × 2 threads)
>   
>   ITERATION 1 (initial positions):
>   T0 → [0]     T1 → [1]     T2 → [2]     T3 → [3]
>   
>   ITERATION 2 (stride by gridSize = 4):
>   T0 → [4]     T1 → [5]     T2 → [6]     T3 → [7]
>   
>   ITERATION 3:
>   T0 → [8]     T1 → [9]     T2 → [10]    T3 → [11]
>   
>   ITERATION 4:
>   T0 → [12]    T1 → [13]    T2 → done    T3 → done
>   
>   PATTERN:
>   ─────────────────────────────────────────────────
>   tid = blockIdx.x * blockDim.x + threadIdx.x
>   stride = blockDim.x * gridDim.x  // Total threads
>   
>   for (i = tid; i < N; i += stride) {
>       process(data[i]);
>   }
>   ─────────────────────────────────────────────────
> ═══════════════════════════════════════════════════════
> ```
> 
> **Key Insight:** Each thread handles elements `tid`, `tid+stride`, `tid+2*stride`, ... until done.

Now let's see the grid-stride loop in action. Notice how we configure the grid **once** and let it handle any size:

### 🔷 CUDA C++ Implementation (Primary)

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
# Demonstrate the limitation
device = cuda.get_current_device()
max_blocks = device.MAX_GRID_DIM_X
threads_per_block = 256
max_elements_naive = max_blocks * threads_per_block

print(f"Max blocks in X dimension: {max_blocks:,}")
print(f"Threads per block: {threads_per_block}")
print(f"Max elements with naive approach: {max_elements_naive:,}")
print(f"That's only {max_elements_naive / 1e9:.2f} billion elements")
print(f"\nModern datasets can have billions of elements!")

---

## Part 3: 2D Grid-Stride Loops

Excellent! You've mastered the 1D pattern. Now let's extend it to 2D - essential for images, matrices, and spatial data.

> 💡 **Concept Card: 2D Grid-Stride Pattern**
> 
> ```
> 🎮 2D GRID-STRIDE: For Images & Matrices
> ═══════════════════════════════════════════════════════
> 
>   Image/Matrix (8×6):              Thread Block (2×2):
>   ┌─────────────────────────┐      ┌───┬───┐
>   │ 0,0  0,1  0,2 ... 0,7   │      │T00│T01│
>   │ 1,0  1,1  1,2 ... 1,7   │      ├───┼───┤
>   │ ...                     │      │T10│T11│
>   │ 5,0  5,1  5,2 ... 5,7   │      └───┴───┘
>   └─────────────────────────┘
>   
>   Each thread strides in BOTH X and Y:
>   
>   T00: (0,0), (0,2), (0,4), (0,6),  ← stride X
>        (2,0), (2,2), (2,4), (2,6),  ← stride Y
>        (4,0), (4,2), (4,4), (4,6)
>        
>   PATTERN:
>   ─────────────────────────────────────────────────
>   x = blockIdx.x * blockDim.x + threadIdx.x
>   y = blockIdx.y * blockDim.y + threadIdx.y
>   strideX = blockDim.x * gridDim.x
>   strideY = blockDim.y * gridDim.y
>   
>   for (row = y; row < height; row += strideY)
>       for (col = x; col < width; col += strideX)
>           idx = row * width + col
>           process(data[idx])
>   ─────────────────────────────────────────────────
> ═══════════════════════════════════════════════════════
> ```
> 
> **Remember:** Outer loop strides rows (Y), inner loop strides columns (X) for coalesced access.

### 🔷 CUDA C++ Implementation (Primary)

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
%%writefile grid_stride_add.cu
// grid_stride_add.cu - Professional grid-stride pattern
#include <stdio.h>
#include <cuda_runtime.h>

// Grid-stride kernel: each thread processes multiple elements
__global__ void gridStrideAdd(const float* a, const float* b, float* out, int n) {
    // Global thread ID
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    // Total number of threads in the grid
    int stride = blockDim.x * gridDim.x;
    
    // Each thread processes elements at tid, tid+stride, tid+2*stride, ...
    for (int i = tid; i < n; i += stride) {
        out[i] = a[i] + b[i];
    }
}

int main() {
    int n = 100000000;  // 100M elements - works with any size!
    size_t size = n * sizeof(float);
    
    // Allocate
    float *d_a, *d_b, *d_out;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_out, size);
    
    // Fixed launch config - works for ANY data size
    int threadsPerBlock = 256;
    int blocksPerGrid = 256;  // Fixed, not dependent on n!
    
    printf("Processing %d elements with %d total threads\n", 
           n, threadsPerBlock * blocksPerGrid);
    printf("Each thread handles ~%d elements\n", 
           n / (threadsPerBlock * blocksPerGrid));
    
    gridStrideAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_out, n);
    cudaDeviceSynchronize();
    
    // Cleanup
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_out);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o grid_stride_add grid_stride_add.cu
!./grid_stride_add

In [ ]:
# Python equivalent for interactive testing
@cuda.jit
def grid_stride_add(a, b, out, n):
    """Grid-stride loop: Each thread processes multiple elements."""
    tid = cuda.grid(1)           # Global thread ID (= blockIdx.x * blockDim.x + threadIdx.x)
    stride = cuda.gridsize(1)    # Total threads (= blockDim.x * gridDim.x)
    
    # Each thread processes elements at tid, tid+stride, tid+2*stride, ...
    for i in range(tid, n, stride):
        out[i] = a[i] + b[i]

### Key Functions

| Function | Returns | Description |
|----------|---------|-------------|
| `cuda.grid(1)` | int | Global thread ID (1D) |
| `cuda.gridsize(1)` | int | Total threads in grid = blocks × threads_per_block |
| `cuda.grid(2)` | (x, y) | Global thread ID (2D) |
| `cuda.gridsize(2)` | (x, y) | Total threads in each dimension |

In [ ]:
# Visualize grid-stride pattern
@cuda.jit
def show_grid_stride(output, n):
    """Store which thread processed each element."""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(tid, n, stride):
        output[i] = tid  # Store thread ID

# Small example
n = 20
blocks = 2
threads = 4
total_threads = blocks * threads

output = np.zeros(n, dtype=np.int32)
d_output = cuda.to_device(output)

show_grid_stride[blocks, threads](d_output, n)
result = d_output.copy_to_host()

print(f"Configuration: {blocks} blocks × {threads} threads = {total_threads} total threads")
print(f"Processing {n} elements")
print(f"\nElement index:  {list(range(n))}")
print(f"Processed by:   {list(result)}")
print(f"\nEach thread processes {n // total_threads} elements (plus remainder)")

---

## Part 3: Benefits of Grid-Stride Loops

### Benefit 1: Handle ANY Data Size

In [ ]:
# Same kernel config works for different sizes
def test_sizes(kernel, sizes, blocks=256, threads=256):
    """Test kernel with various data sizes."""
    for n in sizes:
        a = np.random.rand(n).astype(np.float32)
        b = np.random.rand(n).astype(np.float32)
        out = np.zeros(n, dtype=np.float32)
        
        d_a = cuda.to_device(a)
        d_b = cuda.to_device(b)
        d_out = cuda.to_device(out)
        
        kernel[blocks, threads](d_a, d_b, d_out, n)
        result = d_out.copy_to_host()
        
        # Verify
        expected = a + b
        is_correct = np.allclose(result, expected)
        status = "✓" if is_correct else "✗"
        print(f"{status} N = {n:>12,} | Blocks = {blocks}, Threads = {threads}")

# Test grid-stride with various sizes
print("Grid-Stride Loop (same config for all sizes):")
test_sizes(grid_stride_add, [100, 1000, 10000, 100000, 1000000, 10000000])

### Benefit 2: Tune for Occupancy, Not Data Size

In [ ]:
# Different launch configs, same result
n = 1_000_000
a = np.random.rand(n).astype(np.float32)
b = np.random.rand(n).astype(np.float32)
out = np.zeros(n, dtype=np.float32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_out = cuda.to_device(out)

configs = [
    (32, 64),    # Few threads
    (128, 128),  # Moderate
    (256, 256),  # Typical
    (512, 512),  # High occupancy
]

print(f"Testing different configs with N = {n:,}\n")
for blocks, threads in configs:
    start = time.perf_counter()
    for _ in range(100):
        grid_stride_add[blocks, threads](d_a, d_b, d_out, n)
    cuda.synchronize()
    elapsed = (time.perf_counter() - start) / 100 * 1000
    
    total_threads = blocks * threads
    elements_per_thread = n / total_threads
    print(f"Blocks={blocks:3}, Threads={threads:3} | "
          f"Total={total_threads:>7,} | "
          f"Elem/Thread={elements_per_thread:>6.1f} | "
          f"Time={elapsed:.3f}ms")

### Benefit 3: Better Memory Access Patterns

Grid-stride loops naturally provide **coalesced memory access**:

```
Pass 1: Threads 0,1,2,...,31 access elements 0,1,2,...,31  ← Coalesced!
Pass 2: Threads 0,1,2,...,31 access elements 256,257,...,287  ← Coalesced!
...
```

---

## Part 4: Common Patterns

### Pattern 1: Basic 1D Grid-Stride (CUDA C++)

```cpp
// 1D Grid-Stride Template
__global__ void gridStride1D(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    for (int i = tid; i < n; i += stride) {
        data[i] = data[i] * 2.0f;  // Some operation
    }
}
```

### Pattern 2: 2D Grid-Stride (CUDA C++)

```cpp
// 2D Grid-Stride for images/matrices
__global__ void gridStride2D(float* data, int height, int width) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int strideX = blockDim.x * gridDim.x;
    int strideY = blockDim.y * gridDim.y;
    
    for (int row = y; row < height; row += strideY) {
        for (int col = x; col < width; col += strideX) {
            int idx = row * width + col;
            data[idx] = data[idx] * 2.0f;
        }
    }
}

// Launch with 2D config:
// dim3 threads(16, 16);
// dim3 blocks(32, 32);
// gridStride2D<<<blocks, threads>>>(d_data, height, width);
```

### Pattern 3: Grid-Stride with Local Accumulation (CUDA C++)

```cpp
// Each thread computes partial sum of its elements
__global__ void gridStrideSum(const float* data, float* partialSums, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    float localSum = 0.0f;
    for (int i = tid; i < n; i += stride) {
        localSum += data[i];
    }
    
    // Store partial sum (will need reduction to complete)
    partialSums[tid] = localSum;
}
```

### 🔶 Python/Numba (Optional - Quick Testing)

### 🔷 CUDA C++ 2D Grid-Stride and Accumulation (Primary)

In [ ]:
%%writefile grid_stride_2d.cu
#include <stdio.h>
#include <cuda_runtime.h>

// 2D Grid-Stride for images/matrices
__global__ void gridStride2D(float* data, int height, int width) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int strideX = blockDim.x * gridDim.x;
    int strideY = blockDim.y * gridDim.y;
    
    for (int row = y; row < height; row += strideY) {
        for (int col = x; col < width; col += strideX) {
            int idx = row * width + col;
            data[idx] = data[idx] * 2.0f;
        }
    }
}

// Grid-Stride with Local Accumulation (partial sums)
__global__ void gridStrideSum(const float* data, float* partialSums, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    float localSum = 0.0f;
    for (int i = tid; i < n; i += stride) {
        localSum += data[i];
    }
    
    partialSums[tid] = localSum;
}

int main() {
    // Test 2D Grid-Stride
    const int HEIGHT = 1000;
    const int WIDTH = 1000;
    const int SIZE = HEIGHT * WIDTH;
    const size_t bytes = SIZE * sizeof(float);
    
    float *h_data = (float*)malloc(bytes);
    for (int i = 0; i < SIZE; i++) h_data[i] = 1.0f;
    
    float *d_data;
    cudaMalloc(&d_data, bytes);
    cudaMemcpy(d_data, h_data, bytes, cudaMemcpyHostToDevice);
    
    dim3 threads(16, 16);
    dim3 blocks(32, 32);  // Fixed config works for any image size!
    
    gridStride2D<<<blocks, threads>>>(d_data, HEIGHT, WIDTH);
    
    cudaMemcpy(h_data, d_data, bytes, cudaMemcpyDeviceToHost);
    printf("2D Grid-Stride: data[0] = %.1f (expected 2.0)\n", h_data[0]);
    
    // Test Grid-Stride Sum
    const int N = 1000000;
    float *h_input = (float*)malloc(N * sizeof(float));
    for (int i = 0; i < N; i++) h_input[i] = 1.0f;
    
    float *d_input;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice);
    
    int totalThreads = 256 * 256;
    float *d_partial;
    cudaMalloc(&d_partial, totalThreads * sizeof(float));
    
    gridStrideSum<<<256, 256>>>(d_input, d_partial, N);
    
    float *h_partial = (float*)malloc(totalThreads * sizeof(float));
    cudaMemcpy(h_partial, d_partial, totalThreads * sizeof(float), cudaMemcpyDeviceToHost);
    
    float sum = 0.0f;
    for (int i = 0; i < totalThreads; i++) sum += h_partial[i];
    
    printf("Grid-Stride Sum: %.0f (expected %.0f)\n", sum, (float)N);
    
    cudaFree(d_data);
    cudaFree(d_input);
    cudaFree(d_partial);
    free(h_data);
    free(h_input);
    free(h_partial);
    
    return 0;
}

In [ ]:
!nvcc -o grid_stride_2d grid_stride_2d.cu && ./grid_stride_2d

In [ ]:
# Pattern 1: Basic 1D Grid-Stride (Python)
@cuda.jit
def grid_stride_1d(data, n):
    """Basic 1D grid-stride pattern."""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(tid, n, stride):
        data[i] = data[i] * 2  # Some operation

### Pattern 2: 2D Grid-Stride (Python)

In [ ]:
@cuda.jit
def grid_stride_2d(data, height, width):
    """2D grid-stride for images/matrices."""
    start_x, start_y = cuda.grid(2)
    stride_x, stride_y = cuda.gridsize(2)
    
    for y in range(start_y, height, stride_y):
        for x in range(start_x, width, stride_x):
            data[y, x] = data[y, x] * 2

# Test 2D grid-stride
height, width = 1000, 1000
data = np.random.rand(height, width).astype(np.float32)
d_data = cuda.to_device(data)

threads = (16, 16)
blocks = (32, 32)  # Can be any size, not dependent on image size

grid_stride_2d[blocks, threads](d_data, height, width)
result = d_data.copy_to_host()

print(f"Image: {height}x{width} = {height*width:,} pixels")
print(f"Grid: {blocks[0]*blocks[1]*threads[0]*threads[1]:,} threads")
print(f"Correct: {np.allclose(result, data * 2)}")

### Pattern 3: Grid-Stride with Local Accumulation (Python)

In [ ]:
@cuda.jit
def grid_stride_sum(data, partial_sums, n):
    """Each thread computes partial sum of its elements."""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    local_sum = 0.0
    for i in range(tid, n, stride):
        local_sum += data[i]
    
    # Store partial sum (will need reduction to complete)
    partial_sums[tid] = local_sum

# Test
n = 1_000_000
data = np.random.rand(n).astype(np.float32)
blocks, threads = 256, 256
total_threads = blocks * threads

d_data = cuda.to_device(data)
d_partial = cuda.device_array(total_threads, dtype=np.float32)

grid_stride_sum[blocks, threads](d_data, d_partial, n)
partial = d_partial.copy_to_host()

gpu_sum = partial.sum()  # Final reduction on CPU
cpu_sum = data.sum()

print(f"CPU sum: {cpu_sum:.6f}")
print(f"GPU sum: {gpu_sum:.6f}")
print(f"Match: {np.isclose(cpu_sum, gpu_sum, rtol=1e-4)}")

---

## Part 5: Optimal Launch Configuration

### Guidelines for Choosing Blocks and Threads

In [ ]:
device = cuda.get_current_device()

print("Device Properties:")
print(f"  Max threads per block: {device.MAX_THREADS_PER_BLOCK}")
print(f"  Warp size: {device.WARP_SIZE}")
print(f"  Max blocks per SM: {device.MAX_BLOCK_DIM_X}")
print(f"  Multiprocessors: {device.MULTIPROCESSOR_COUNT}")

# Recommended config
threads = 256  # Multiple of warp size (32)
blocks = device.MULTIPROCESSOR_COUNT * 4  # Enough for good occupancy

print(f"\nRecommended starting config:")
print(f"  Threads per block: {threads}")
print(f"  Blocks: {blocks}")
print(f"  Total threads: {blocks * threads:,}")

---

## Why Grid-Stride is Superior

Now let's compare the approaches side-by-side to understand why grid-stride is the professional choice:

| Aspect | Naive (1 thread = 1 element) | Grid-Stride Loop |
|--------|------------------------------|------------------|
| **Grid sizing** | Must match data size | Fixed, tuned once |
| **Large data** | Huge grids, overhead | Same small grid |
| **Small data** | Underutilized threads | Same config works |
| **Occupancy tuning** | Tied to data size | Independent tuning |
| **Code reuse** | Different configs | One kernel fits all |
| **Memory coalescing** | Natural | Natural |

> 🔑 **Professional Insight:** Grid-stride loops let you tune your kernel launch configuration for **maximum occupancy** on your specific GPU, regardless of problem size.

---

## Part 4: Performance Analysis

Let's measure the real-world impact of grid-stride loops:

---

## Part 6: Benchmarking

In [ ]:
def benchmark_add(n, iterations=100):
    """Compare naive vs grid-stride for vector addition."""
    a = np.random.rand(n).astype(np.float32)
    b = np.random.rand(n).astype(np.float32)
    out = np.zeros(n, dtype=np.float32)
    
    d_a = cuda.to_device(a)
    d_b = cuda.to_device(b)
    d_out = cuda.to_device(out)
    
    # Naive config: one thread per element
    threads_naive = 256
    blocks_naive = (n + threads_naive - 1) // threads_naive
    
    # Grid-stride config: fixed size
    threads_gs = 256
    blocks_gs = 256
    
    # Warmup
    naive_add[blocks_naive, threads_naive](d_a, d_b, d_out, n)
    grid_stride_add[blocks_gs, threads_gs](d_a, d_b, d_out, n)
    cuda.synchronize()
    
    # Benchmark naive
    start = time.perf_counter()
    for _ in range(iterations):
        naive_add[blocks_naive, threads_naive](d_a, d_b, d_out, n)
    cuda.synchronize()
    naive_time = (time.perf_counter() - start) / iterations * 1000
    
    # Benchmark grid-stride
    start = time.perf_counter()
    for _ in range(iterations):
        grid_stride_add[blocks_gs, threads_gs](d_a, d_b, d_out, n)
    cuda.synchronize()
    gs_time = (time.perf_counter() - start) / iterations * 1000
    
    # NumPy baseline
    start = time.perf_counter()
    for _ in range(iterations):
        _ = a + b
    numpy_time = (time.perf_counter() - start) / iterations * 1000
    
    return naive_time, gs_time, numpy_time, blocks_naive

print(f"{'N':>12} | {'Naive (ms)':>10} | {'Grid-Stride':>11} | {'NumPy':>10} | {'GPU Speedup':>11}")
print("-" * 65)

for n in [10_000, 100_000, 1_000_000, 10_000_000]:
    naive_t, gs_t, np_t, blocks = benchmark_add(n)
    speedup = np_t / gs_t
    print(f"{n:>12,} | {naive_t:>10.3f} | {gs_t:>11.3f} | {np_t:>10.3f} | {speedup:>10.1f}x")

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these grid-stride loop exercises in CUDA C++.

In [ ]:
%%writefile grid_stride_exercises.cu
// grid_stride_exercises.cu - Grid-stride loop exercises
#include <stdio.h>
#include <cuda_runtime.h>
#include <math.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA Error: %s\n", cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

// =============================================================================
// Exercise 1: Vector Scaling with Grid-Stride Loop
// =============================================================================

__global__ void vectorScale(float* data, float scalar, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    // Grid-stride loop
    for (int i = idx; i < n; i += stride) {
        data[i] *= scalar;
    }
}

// =============================================================================
// Exercise 2: Vector Square Root with Grid-Stride Loop
// =============================================================================

__global__ void vectorSqrt(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    for (int i = idx; i < n; i += stride) {
        data[i] = sqrtf(data[i]);
    }
}

// =============================================================================
// Exercise 3: Conditional Processing (threshold filter)
// =============================================================================

__global__ void thresholdFilter(const float* input, float* output, 
                                 float threshold, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    for (int i = idx; i < n; i += stride) {
        output[i] = (input[i] > threshold) ? input[i] : 0.0f;
    }
}

// =============================================================================
// Exercise 4: 2D Brightness Adjustment with Grid-Stride
// =============================================================================

__global__ void adjustBrightness2D(float* image, float delta, 
                                    int width, int height) {
    // 2D grid-stride loop
    int startX = blockIdx.x * blockDim.x + threadIdx.x;
    int startY = blockIdx.y * blockDim.y + threadIdx.y;
    int strideX = blockDim.x * gridDim.x;
    int strideY = blockDim.y * gridDim.y;
    
    for (int y = startY; y < height; y += strideY) {
        for (int x = startX; x < width; x += strideX) {
            int idx = y * width + x;
            float val = image[idx] + delta;
            image[idx] = fminf(fmaxf(val, 0.0f), 1.0f);  // Clamp to [0,1]
        }
    }
}

// =============================================================================
// Test harness
// =============================================================================

int main() {
    printf("=== Grid-Stride Loop Exercises ===\n\n");
    
    // Exercise 1: Vector Scaling
    printf("Exercise 1: Vector Scaling\n");
    printf("-" "-------------------------\n");
    {
        const int N = 100000000;  // 100M elements - more than any grid can handle
        float* h_data = (float*)malloc(N * sizeof(float));
        for (int i = 0; i < N; i++) h_data[i] = (float)(i + 1);
        
        float* d_data;
        CUDA_CHECK(cudaMalloc(&d_data, N * sizeof(float)));
        CUDA_CHECK(cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice));
        
        // Use limited grid - grid-stride handles the rest
        int threads = 256;
        int blocks = 256;  // Only 65K threads for 100M elements!
        
        vectorScale<<<blocks, threads>>>(d_data, 3.0f, N);
        CUDA_CHECK(cudaDeviceSynchronize());
        
        CUDA_CHECK(cudaMemcpy(h_data, d_data, N * sizeof(float), cudaMemcpyDeviceToHost));
        
        printf("Scaled 100M elements with only %d threads\n", blocks * threads);
        printf("Sample: data[0]=%.0f, data[4]=%.0f (expected: 3, 15)\n\n", 
               h_data[0], h_data[4]);
        
        cudaFree(d_data);
        free(h_data);
    }
    
    // Exercise 2: Vector Sqrt
    printf("Exercise 2: Vector Square Root\n");
    printf("-" "------------------------------\n");
    {
        const int N = 10000;
        float* h_data = (float*)malloc(N * sizeof(float));
        for (int i = 0; i < N; i++) h_data[i] = (float)((i + 1) * (i + 1));  // Perfect squares
        
        float* d_data;
        CUDA_CHECK(cudaMalloc(&d_data, N * sizeof(float)));
        CUDA_CHECK(cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice));
        
        vectorSqrt<<<32, 256>>>(d_data, N);
        CUDA_CHECK(cudaDeviceSynchronize());
        
        CUDA_CHECK(cudaMemcpy(h_data, d_data, N * sizeof(float), cudaMemcpyDeviceToHost));
        
        printf("Sample: sqrt(1)=%.0f, sqrt(4)=%.0f, sqrt(9)=%.0f\n\n", 
               h_data[0], h_data[1], h_data[2]);
        
        cudaFree(d_data);
        free(h_data);
    }
    
    // Exercise 3: Threshold Filter
    printf("Exercise 3: Threshold Filter\n");
    printf("-" "----------------------------\n");
    {
        const int N = 1000;
        float* h_input = (float*)malloc(N * sizeof(float));
        float* h_output = (float*)malloc(N * sizeof(float));
        for (int i = 0; i < N; i++) h_input[i] = (float)(i % 100) / 100.0f;
        
        float *d_input, *d_output;
        CUDA_CHECK(cudaMalloc(&d_input, N * sizeof(float)));
        CUDA_CHECK(cudaMalloc(&d_output, N * sizeof(float)));
        CUDA_CHECK(cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice));
        
        thresholdFilter<<<4, 256>>>(d_input, d_output, 0.5f, N);
        CUDA_CHECK(cudaDeviceSynchronize());
        
        CUDA_CHECK(cudaMemcpy(h_output, d_output, N * sizeof(float), cudaMemcpyDeviceToHost));
        
        int above = 0, zeroed = 0;
        for (int i = 0; i < N; i++) {
            if (h_output[i] > 0) above++; else zeroed++;
        }
        printf("Threshold=0.5: %d above, %d zeroed\n\n", above, zeroed);
        
        cudaFree(d_input); cudaFree(d_output);
        free(h_input); free(h_output);
    }
    
    // Exercise 4: 2D Brightness
    printf("Exercise 4: 2D Brightness Adjustment\n");
    printf("-" "------------------------------------\n");
    {
        const int W = 4096, H = 4096;
        float* h_image = (float*)malloc(W * H * sizeof(float));
        for (int i = 0; i < W * H; i++) h_image[i] = 0.3f;  // Dark image
        
        float* d_image;
        CUDA_CHECK(cudaMalloc(&d_image, W * H * sizeof(float)));
        CUDA_CHECK(cudaMemcpy(d_image, h_image, W * H * sizeof(float), cudaMemcpyHostToDevice));
        
        // Small grid for large image
        dim3 block(16, 16);
        dim3 grid(16, 16);  // Only 256 blocks for 16M pixels
        
        adjustBrightness2D<<<grid, block>>>(d_image, 0.5f, W, H);
        CUDA_CHECK(cudaDeviceSynchronize());
        
        CUDA_CHECK(cudaMemcpy(h_image, d_image, W * H * sizeof(float), cudaMemcpyDeviceToHost));
        
        printf("Adjusted 4096x4096 image with %d blocks\n", 16*16);
        printf("Sample: pixel=%.1f (was 0.3, +0.5 = 0.8)\n", h_image[0]);
        
        cudaFree(d_image);
        free(h_image);
    }
    
    printf("\n=== All exercises complete! ===\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o grid_stride_exercises grid_stride_exercises.cu && ./grid_stride_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: Vector Scaling

In [ ]:
# Exercise 1: Vector Scaling (Python)
@cuda.jit
def vector_scale(data, scalar, n):
    """Multiply every element by scalar: data[i] *= scalar"""
    # TODO: Your implementation here
    pass

# Test
# data = np.array([1, 2, 3, 4, 5], dtype=np.float32)
# Expected after scale by 3: [3, 6, 9, 12, 15]

### Exercise 2: Vector Square Root (Python)

In [ ]:
# TODO: Implement grid-stride sqrt
@cuda.jit
def vector_sqrt(input_data, output_data, n):
    """Compute sqrt of every element."""
    # Hint: Use math.sqrt(x) inside the kernel
    pass

# Test with input = [1, 4, 9, 16, 25]
# Expected output = [1, 2, 3, 4, 5]

### Exercise 3: Conditional Processing (Python)

In [ ]:
# TODO: Double only positive values
@cuda.jit
def double_positives(data, n):
    """Double the value of positive elements, leave others unchanged."""
    # Your implementation here
    pass

# Test with input = [-2, -1, 0, 1, 2]
# Expected output = [-2, -1, 0, 2, 4]

### Exercise 4: 2D Brightness Adjustment (Python)

In [ ]:
# TODO: Adjust image brightness
@cuda.jit
def adjust_brightness(image, factor, height, width):
    """Multiply all pixel values by factor, clamping to [0, 1]."""
    # Use 2D grid-stride pattern
    # Clamp: max(0, min(1, value))
    pass

# Test with a 100x100 image

---

## 📝 Key Takeaways

### Quick Reference Card: Grid-Stride Loops

```
┌─────────────────────────────────────────────────────────────────┐
│  GRID-STRIDE LOOP PATTERN                                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1D PATTERN:                                                    │
│  ───────────────────────────────────────────────────────────    │
│  int tid = blockIdx.x * blockDim.x + threadIdx.x;               │
│  int stride = blockDim.x * gridDim.x;                           │
│                                                                 │
│  for (int i = tid; i < n; i += stride) {                        │
│      data[i] = process(data[i]);                                │
│  }                                                              │
│                                                                 │
│  2D PATTERN:                                                    │
│  ───────────────────────────────────────────────────────────    │
│  int x = blockIdx.x * blockDim.x + threadIdx.x;                 │
│  int y = blockIdx.y * blockDim.y + threadIdx.y;                 │
│  int strideX = blockDim.x * gridDim.x;                          │
│  int strideY = blockDim.y * gridDim.y;                          │
│                                                                 │
│  for (int row = y; row < height; row += strideY)                │
│      for (int col = x; col < width; col += strideX) {           │
│          int idx = row * width + col;                           │
│          data[idx] = process(data[idx]);                        │
│      }                                                          │
│                                                                 │
│  RECOMMENDED LAUNCH CONFIG:                                     │
│  ───────────────────────────────────────────────────────────    │
│  int threadsPerBlock = 256;        // Multiple of 32 (warp)     │
│  int blocksPerGrid = numSMs * 4;   // Good occupancy            │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### ✅ What You Achieved Today

| Skill | Status |
|-------|--------|
| Identified naive kernel limitations | ✅ |
| Implemented 1D grid-stride loops | ✅ |
| Extended to 2D grid-stride loops | ✅ |
| Configured optimal launch parameters | ✅ |

### 🧠 Key Benefits to Remember

1. ✓ **Handle any data size** with fixed launch config
2. ✓ **Tune occupancy independently** of data
3. ✓ **Natural coalesced access** pattern
4. ✓ **Professional, reusable** code template

---

## 🚀 What's Next?

**Day 2: Element-wise Vector Operations** — We'll use grid-stride loops as the foundation for all our vector operations!

| Preview Topic | What You'll Learn |
|---------------|-------------------|
| Basic arithmetic | add, sub, mul, div on vectors |
| Math functions | sqrt, exp, log, sin, cos |
| Activation functions | ReLU, sigmoid, tanh |
| Compound operations | Combining multiple ops |

> 💡 **Tomorrow's Hook:** What's the fastest way to apply the same operation to millions of independent elements? (Hint: you already know the pattern!)